In [1]:

import tensorflow as tf
import cv2
import pyautogui
import numpy as np
from keras_cv.models import YOLOV8Detector
from keras_cv import bounding_box
from keras_cv import visualization
import os
from tensorflow.keras.models import load_model
import keras_cv

2023-09-27 12:24:47.980176: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 12:24:50.558096: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


/home/tomas/miniconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#HYPERPARAMETERS 

SPLIT_RATIO = 0.2
BATCH_SIZE = 4
LEARNING_RATE = 0.001
EPOCH = 5
GLOBAL_CLIPNORM = 10.0

In [3]:
# Check TensorFlow Version
print("TensorFlow Version:", tf.__version__)

# Check for GPU availability
print("GPU Available:", tf.test.is_gpu_available())

# List the names of available GPUs
print("GPU Device Names:", tf.config.list_physical_devices('GPU'))


TensorFlow Version: 2.13.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU Available: True
GPU Device Names: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-09-27 12:25:03.229053: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 12:25:03.241864: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 12:25:03.242164: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:

# Load the model
custom_objects = {'YOLOV8Detector': YOLOV8Detector}
loaded_yolo = load_model('./goblin-detector-model_v1', custom_objects=custom_objects, compile=False)

# Manually compile the model
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    global_clipnorm=GLOBAL_CLIPNORM,
)

loaded_yolo.compile(
    optimizer=optimizer,
    classification_loss="binary_crossentropy",
    box_loss="ciou"
)

2023-09-27 12:25:03.965634: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 12:25:03.966959: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 12:25:03.967333: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [10]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    return image

def get_test_data(directory="./test_data"):
    files = os.listdir(directory)
    image_paths = [os.path.join(directory, f) for f in files if f.endswith(".jpg")]
    data = tf.data.Dataset.from_tensor_slices(image_paths)
    data = data.map(load_image)
    return data

def visualize_detections(model, dataset, bounding_box_format):
    images = next(iter(dataset.take(1)))
    
    print(f"test images shape: {images.shape}")
    y_pred = model.predict(images)
    print(f"predictions len: {len(y_pred)}")
    
    # Assuming each box in y_pred['boxes'] is assigned the "Goblin" class
    y_pred['classes'] = tf.constant([[0] * y_pred['boxes'].shape[1] for _ in range(len(images))])

    y_pred = bounding_box.to_ragged(y_pred)
    
    y_true = [[] for _ in range(len(images))]
    
    # Define the Goblin class mapping
    class_mapping = {0: "Goblin"}
    
    visualization.plot_bounding_box_gallery(
        images,
        value_range=(0, 255),
        bounding_box_format=bounding_box_format,
        y_true=y_true,
        y_pred=y_pred,
        scale=4,
        rows=2,
        cols=2,
        show=True,
        font_scale=0.7,
        class_mapping=class_mapping
    )

In [11]:
loaded_yolo.input_shape

(None, None, None, 3)

In [12]:
#check if detections are ok with test data
resizing = keras_cv.layers.JitteredResize(
    target_size=(640, 640),
    scale_factor=(0.75, 1.3),
    bounding_box_format="xyxy",
)
test_data = get_test_data()

test_data  = test_data.shuffle(BATCH_SIZE * 4)
test_data = test_data.ragged_batch(BATCH_SIZE, drop_remainder=True)
test_data = test_data.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)

# Visualize detections on the test dataset
visualize_detections(loaded_yolo, dataset=test_data, bounding_box_format="xyxy")

test images shape: (4, 640, 640, 3)
1/1 [==============================] - 1s 527ms/step
predictions len: 4


ValueError: Expected `boxes` and `classes` to have matching dimensions on the first two axes when operating in batched mode. Got `boxes.shape=(4, 100, 4)`, `classes.shape=(4,)`.

In [ ]:


def start_detecting():


    # Yu can use pyautogui.size() to get the screen size if needed
    # screen_width, screen_height = pyautogui.size()

    # while True:
    #     # Capture the screen
    #     screenshot = pyautogui.screenshot(region=(0, 0, screen_width, screen_height))
    #     frame = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)

    #     # Prepare the frame for the model
    #     input_frame = cv2.resize(frame, (640, 640))  # Assuming your YOLO model takes 416x416 images
    #     input_frame = np.expand_dims(input_frame, axis=0)

    #     # Predict
    #     y_pred_dict = loaded_yolo.predict(input_frame)

    #     # Get boxes, confidence, and classes from prediction
    #     boxes = y_pred_dict['boxes'][0]  # First index 0 assumes a batch size of 1
    #     confidence = y_pred_dict['confidence'][0]
    #     classes = y_pred_dict['classes'][0]

    #     # Filter out detections with confidence -1 or classes -1
    #     for box, conf, cls in zip(boxes, confidence, classes):
    #         if conf == -1 or cls == -1:
    #             continue

    #         # Draw rectangle
    #         x1, y1, x2, y2 = map(int, box)  # Convert to integers
    #         cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
    #         # Add label
    #         cv2.putText(frame, str(cls), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    #     # Show the frame
    #     cv2.imshow('Object Detection', frame)

    #     # Break the loop if 'q' is pressed
    #     if cv2.waitKey(1) & 0xFF == ord('q'):
    #         break

    # cv2.destroyAllWindows()





In [ ]:


gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Num GPUs Available: {len(gpus)}")
    for gpu in gpus:
        print(f"GPU: {gpu}")
    start_detecting()
else:
    print("No GPUs are available!")

Num GPUs Available: 1
GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2023-09-27 12:13:09.785798: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 62914560 exceeds 10% of free system memory.


images shape: (4, 640, 640, 3)


2023-09-27 12:13:21.142954: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2023-09-27 12:13:21.580377: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-27 12:13:21.582571: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-27 12:13:21.585029: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:109] Couldn't get ptxas version : FAILED_PRECONDITION: Couldn't get ptxas/nvlink version string: INTERNAL: Couldn't invoke ptxas --version
2023-09-27 12:13:21.586169: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-27 12:13:21.588812: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This mes

1/1 [==============================] - 15s 15s/step
predictions: {'boxes': array([[[308.5429   , 491.65555  , 384.2222   , 594.4419   ],
        [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
        [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
        ...,
        [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
        [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
        [ -1.       ,  -1.       ,  -1.       ,  -1.       ]],

       [[401.50073  ,  80.7566   , 459.5499   , 160.3955   ],
        [142.69434  ,   1.8485489, 189.73068  ,  56.85008  ],
        [ -0.9805565, 126.825485 ,  47.368233 , 217.50894  ],
        ...,
        [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
        [ -1.       ,  -1.       ,  -1.       ,  -1.       ],
        [ -1.       ,  -1.       ,  -1.       ,  -1.       ]],

       [[382.80878  , 361.44843  , 445.2689   , 459.3841   ],
        [207.82317  , 265.69373  , 274.7027   , 334.79028  ],
        [ 97.5333   , 158.1

ImportError: plot_bounding_box_gallery requires the `matplotlib` package. Please install the package using `pip install matplotlib`.